In [ ]:
import logging
import sys
import pathlib
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import pandas as pd
import cv2
import tifffile
import matplotlib.pyplot as plt
import staintools

sys.path.append('../src')
import settings
import annotation_utils

In [ ]:
with open(settings.DATA / 'imaging_measurements.json') as f:
    imaging_measurements = json.load(f)

# Raw training and test data
df_hubmap_hpa_organ_segmentation_metadata = pd.concat((
    pd.read_csv(settings.DATA / 'train_metadata.csv'),
    pd.read_csv(settings.DATA / 'test_metadata.csv')
), axis=0, ignore_index=True)

In [ ]:
imaging_measurements

In [ ]:
def visualize_image(image, organ, metadata, path=None):

    """
    Visualize image along with its annotations
    
    Parameters
    ----------
    image (path-like str or numpy.ndarray of shape (height, width, 3)): Image path relative to root/data or image array
    path (path-like str or None): Path of the output file or None (if path is None, plot is displayed with selected backend)
    """

    if isinstance(image, pathlib.Path) or isinstance(image, str):
        # Read image from the given path
        image_path = image
        image = tifffile.imread(str(image_path))

    elif isinstance(image, np.ndarray):
        title = ''

    else:
        # Raise TypeError if image argument is not an array-like object or a path-like string
        raise TypeError('Image is not an array or path.')
        
    alpha = 0.15
    tissue_thickness_scale_factor = imaging_measurements['hubmap']['tissue_thickness'][organ] - imaging_measurements['hpa']['tissue_thickness'][organ]
    image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV).astype(np.float32)
    image_hsv[:, :, 1] *= (1 + (alpha * tissue_thickness_scale_factor))
    image_hsv[:, :, 2] *= (1 - (alpha * tissue_thickness_scale_factor))
    image_hsv = image_hsv.astype(np.uint8)
    image_scaled = cv2.cvtColor(image_hsv, cv2.COLOR_HSV2RGB)

    pixel_size_scale_factor = imaging_measurements['hpa']['pixel_size'][organ] / imaging_measurements['hubmap']['pixel_size'][organ]
    image_resized = cv2.resize(
        image_scaled,
        dsize=None,
        fx=pixel_size_scale_factor,
        fy=pixel_size_scale_factor,
        interpolation=cv2.INTER_CUBIC
    )
    image_resized = cv2.resize(
        image_resized,
        dsize=(
            image.shape[1],
            image.shape[0]
        ),
        interpolation=cv2.INTER_CUBIC
    )
    
    image = staintools.LuminosityStandardizer.standardize(image)
    image_resized = staintools.LuminosityStandardizer.standardize(image_resized)
        
    fig, axes = plt.subplots(figsize=(36, 20), ncols=2)
    
    axes[0].imshow(image)
    axes[1].imshow(image_resized)

    for i in range(2):
        axes[i].set_xlabel('')
        axes[i].set_ylabel('')
        axes[i].tick_params(axis='x', labelsize=15, pad=10)
        axes[i].tick_params(axis='y', labelsize=15, pad=10)

    axes[0].set_title('Image', size=25, pad=15)
    axes[1].set_title('Image with HuBMAP Imaging Measurements', size=25, pad=15)

    fig.suptitle(
        f'''
        Image ID {metadata["id"]} - {metadata["organ"]} - {metadata["data_source"]} - {metadata["age"]} - {metadata["sex"]}
        Image Shape: {metadata["image_height"]}x{metadata["image_width"]} - Pixel Size: {metadata["pixel_size"]}µm - Tissue Thickness: {metadata["tissue_thickness"]}µm
        ''',
        fontsize=50
    )

    if path is None:
        plt.show()
    else:
        plt.savefig(path)
        plt.close(fig)


In [ ]:
for idx, row in df_sample.iterrows():
    
    image = tifffile.imread(row['image_filename'])
    organ = row['organ']
    
    visualize_image(
        image=image,
        organ=organ,
        metadata=row.to_dict()
    )
    